In [25]:
import pandas as pd 
import numpy as np
from regularized_var import VAR, MinnesotaVAR

In [26]:
# load sample data
X = pd.read_csv('data.csv')
X['date'] = pd.to_datetime(X['date'])
X.set_index('date', inplace=True)

X.head()[['ct2_bps', 'ct5_bps', 'ct10_bps', 'ct30_bps']]

,ct2_bps,ct5_bps,ct10_bps,ct30_bps
date,,,,
1977-02-15,6.04,6.72,7.36,7.70
1977-02-16,6.02,6.70,7.34,7.67
1977-02-17,5.99,6.67,7.26,7.67
1977-02-18,6.06,6.82,7.41,7.76
1977-02-21,6.06,6.82,7.41,7.76


In [12]:
# define variables to include. Note model assumes already differenced/stationary
ENDOG = [
    'ct2_bps_ret',
    'ct5_bps_ret',
    'ct10_bps_ret',
    'ct30_bps_ret'
]

In [27]:
# standardize 
df_norm = (X[ENDOG] - X[ENDOG].mean()) / X[ENDOG].std()
df_norm.index = X.index
df_norm.head()

,ct2_bps_ret,ct5_bps_ret,ct10_bps_ret,ct30_bps_ret
date,,,,
1977-02-15,0.002254,0.002988,0.003402,0.003439
1977-02-16,-0.251366,-0.261470,-0.280843,-0.466342
1977-02-17,-0.378176,-0.393699,-1.133579,0.003439
1977-02-18,0.889924,1.986423,2.135242,1.412782
1977-02-21,0.002254,0.002988,0.003402,0.003439


**Ridge VAR**

*l2 regularization, applied evenly on parameters*

In [36]:
# set up model
model = VAR(
    n_lags=5,
    alpha=1.0, # strength of regularization, similar to sklearn's implementation of alpha in Ridge (0->inf; higher = stronger shrinkage)
    include_const=False
)

# fit
res = model.fit(df_norm[ENDOG])

In [37]:
# out of sample prediction
res.predict(steps=5)

,ct2_bps_ret,ct5_bps_ret,ct10_bps_ret,ct30_bps_ret
step,,,,
1,0.031630,0.028149,0.024083,0.014327
2,0.008062,0.002752,0.005792,0.008569
3,-0.005497,-0.002358,-0.004422,-0.003651
4,0.001261,0.000516,-0.002299,-0.000659
5,0.004785,0.002240,0.004701,0.003782


**Minnesota VAR**

*l2 regularization, applied akin to Minnesota/Litterman prior*

In [38]:
# set up model
model = MinnesotaVAR(
    n_lags=5,
    alpha_own=1.0, # strength of regularization on X(i)'s own lags
    alpha_cross=2.0, # strength of regularization on X(i)'s non-X(i) lags
    power=2.0, # weight decay -> longer lags get stronger shrinkage
    include_const=False
)

# fit
res = model.fit(df_norm[ENDOG])

In [39]:
# out of sample prediction
res.predict(steps=5)

,ct2_bps_ret,ct5_bps_ret,ct10_bps_ret,ct30_bps_ret
step,,,,
1,0.045614,0.041041,0.033299,0.019667
2,0.012248,0.003255,0.005764,0.008772
3,-0.004249,-0.001705,-0.003047,-0.002275
4,0.001187,0.000210,-0.001948,-0.000370
5,0.002576,0.001020,0.002313,0.001391
